## Hidden Markov Model

Hidden Markov Model은 관측 시퀀스에 대한 분포를 표현하는 확률적 모델이다. HMM은 가장 단순한 Dynamic Bayesian Network이고, 금융을 포함한 많은 응용 분야에서 강력한 모델임이 입증됐다. 이제까지는 주로 IID 관측만을 가정했다. 물론, 금융 모델링은 많은 경우 관측이 이전 관측, 그리고 그 이전 관측에만 의존하는 마르코프 설정에서 수행된다. HMM은 확률적 모델링의 중요한 아이디어를 함축하고 있으므로 여기서 간단히 살펴본다. 특히 이들은 은닉 변수와 전환(switching)을 이해하는 데 통찰력을 제공한다. Gating과 같은 알고리즘을 사용하는 RNN, LSTM과 같은 동적 순환 신경망은 암묵적인 모델링 단계임으로, 금융의 국면 전환에 필요한 만큼 명시적으로 제어할 수 없다.

이제 시점 $t$에 이산 상태 $s_t$가 관측자에게 보이지 않는다고 가정하자. 이 설정은 은닉 변수가 IID라고 가정하는 Mixtual Model과는 다르다. 시점 $t$ 관측 $y_t$는 다른 모든 시점의 상태와 독립이라고 가정한다. Markov 특성에 의해 상태 시퀀스 $s := \{s_t\}_{i=1}^T$와 관측 시퀀스 $y = \{y_t\}_{t=1}^T$의 결합 확률은 전이 확률 밀도 $p(s_t|s_{t-1})$과 출력 확률 밀도 $p(y_t|s_t)$의 곱으로 구할 수 있다.

$$p(s, y) = p(s_1)p(y_1 | s_1) \prod_{t=2}^T p(s_t | s_{t-1})p(y_t | s_t)$$

In [1]:
import numpy as np
import pandas as pd

우선 모든 상태 $s_t \in \{1, \dots, K\}$와 모든 시점에 대해 소위 전방과 후방 확률을 각각 도입한다.

$$F_t(s) := \mathbb{P}(s_t = s, y_{1:t}), B_t(s) := p(y_{t_1:T} | s_t = s)$$

여기서 관항에 따라 $B_T(s) = 1$이다. 모든 $t \in \{1, \dots, T\}$와 모든 $r, s \in \{1, \dots, K\}$에 대해 다음을 얻는다.

$$\mathbb{P}(s_t = s, y) = F_t(s)B_t(s)$$

그리고 전방과 후방 확률을 결합하면 다음을 산출할 수 있다.

$$\mathbb{P}(s_{t-1} = r, s_t = s, y) = F_{t-1}(r) \mathbb{P}(s_t = s|s_{t-1} = r)p(y_t | s_t = s)B_t(s)$$

Baum-Welch Algorithm으로도 알려져 있는 이 forward-backward algorithm은 EM algorithm 클래스에 속하는 HMM을 적합화하기 위한 비지도학습 알고리즘이다.

In [2]:
def forward(obs, start_prob, trans_prob, em_prob) :
    num_obs = len(obs)
    num_states = len(start_prob)
    F = np.zeros((num_obs, num_states))
    
    # calculate initial state probability and conditional probability of first observation
    F[0, :] = start_prob * em_prob[:, obs[0]]
    
    # calculate other F values recursively
    for t in range(1, num_obs) :
        for j in range(num_states) :
            F[t, j] = np.dot(F[t-1, :], trans_prob[:, j])
            
    return F

def backward(obs, trans_prob, em_prob) :
    num_obs = len(obs)
    num_states = len(trans_prob)
    B = np.ones((num_obs, num_states))
    
    # calculate other B values recursively
    for t in range(num_obs-2, -1, -1) :
        for i in range(num_states) :
            B[t, i] = np.sum(trans_prob[:, i] * em_prob[:, obs[t+1]] * B[t+1, :])
            
    return B

In [3]:
start_prob = np.array([0.5, 0.5])
trans_prob = np.array([
    [0.7, 0.3],
    [0.4, 0.6]
])
em_prob = np.array([
    [0.1, 0.4, 0.5],
    [0.6, 0.3, 0.1]
]) # Emission probabilities
obs = np.array([0, 1, 2, 0, 1]) # Index of observations

In [4]:
F = forward(obs, start_prob, trans_prob, em_prob)
B = backward(obs, trans_prob, em_prob)
print("Forward probabilities:\n", F)
print("Backward probabilities:\n", B)

Forward probabilities:
 [[0.05     0.3     ]
 [0.155    0.195   ]
 [0.1865   0.1635  ]
 [0.19595  0.15405 ]
 [0.198785 0.151215]]
Backward probabilities:
 [[0.013808 0.008772]
 [0.0398   0.0222  ]
 [0.1      0.12    ]
 [0.4      0.3     ]
 [1.       1.      ]]


### 1. Viterbi Algorithm

특정 은닉 상태의 실현 확률을 찾는 것에 더해 가장 가능한 시퀀스 실현을 찾을 수 있다. 이 시퀀스는 Viterbi algorithm을 사용해 추정할 수 있다. 다시 $T$개의 관측 시퀀스를 관측한다고 가정하자.

$$\mathbf{y} = \{y_1, \dots, y_T\}$$

그러나 각 $1 \leq t \leq T, y_t \in O$에 대해 $O = \{o_1, o_2, \dots, o_N\}, N \in \mathbb{N}$은 이제 어떤 관측 공간에 있다. 각 $1 \leq t \leq T$에 대해 $y_t$는 (은닉)상태 $s_t \in \mathcal{S}$에 의해 주도되며, 여기서 $\mathcal{S} := \{ \int_1, \dots, \int_K\}, K \in \mathbb{N}$은 어떤 상태 공간이다. 예를 들어 $y_t$는 어떤 회사채의 신용 등급이고 $s_t$는 관련 산업 섹터의 전체적인 건전성과 같은 어떤 잠재 변수를 가리킬 수 있다.

주어진 $\mathbf{y}$에 대해 가장 가능한 은닉 상태의 시퀀스는 무엇일까?

$$\mathbf{x} = \{x_1, x_2,\ dots, x_T\}$$

이 문제에 답하고자 추가적인 변수들을 도입한다. 첫째, 반드시 다음과 같은 초기 확률 집합이 주어져야만 한다.

$$\mathcal{\pi} = \{\pi_1, \dots, \pi_K\}$$

따라서 $\pi_i $는 $s_1 = \int_i, 1 \leq i \leq K$일 확률이다. 또한 원소 $A_{ij}, 1 \leq i, j \leq K$가 상태 $\int_i$에서 상태 $\int_j$로 전이하는 전이 확률인 전이 행렬(transition matrix) $A \in \mathbb{R}^{K \times K}$을 설정할 필요가 있다. 마지막으로 원소 $B_{ij}, 1 \leq i \leq K, 1 \leq j \leq N$가 상태 $\int_i$에서 $o_j$를 관측하는 확률인 출력 행렬(emission matrix) $B \in \mathbb{R}^{K \times N}$을 필요로 한다.

관측 시퀀스 $y = \{y_1, \dots, y_T\}$를 산출하는 가장 가능한 상태 시퀀스 $s$는 다음의 재귀 관계를 만족한다.

$$
\begin{align}
V_{1,k} = & \mathbb{P}(y_1 | s_1 = \int_k)\cdot \pi_k \\ 
V_{t,k} = & \max_{1 \leq i \leq K} \left( \mathbb{P}\left(y_t | s_t = \int_k\right) \cdot A_{ik} \cdot V_{t-1, i}\right)
\end{align}
$$

여기서 $V_{t,k}$는 $s_t = \int_k$일 때의 가장 가능한 상태 시퀀스 $\{s_1, \dots, s_t\}$의 확률이다.

$$V_{t,k} = \mathbb{P}\left(s_1, \dots, s_t, y_1, \dots, y_t| s_t = \int_k\right)$$

실제 Viterbi Path는 각 단계에서 두 번째 식에 어떤 상태 인덱스 $i$가 사용됐는가를 추적함으로써 구할 수 있다. 이제 $\xi(k, t)$를 $t > 1$이면 $V_{t,k}$또는 $t=1$이면 $k$를 계산하는 데 사용된 $i$의 값을 반환하는 함수라 하자. 그러면 다음과 같다.

$$
\begin{align}
s_T = & \int_{\arg \max_{1 \leq i \leq K}} \left( V_{T,k}\right) \\ 
s_{t-1} = & \int_{\xi (s_t, t)}& 
\end{align}
$$

Viterbi Algorithm은 관측 시퀀스와 알려진 전이와 출력 행렬을 포함한 모델 설정이 주어졌을 때 은닉 상태의 가장 가능한 시퀀스를 결정한다. 이들 행렬이 알려져 있으므로 Baum-Welch Algorithm을 사용할 필요는 없다. 알려져 있지 않다면 Baum-Welch Algorithm을 사용해야 한다.

In [5]:
def viterbi(obs, states, start_prob, trans_prob, em_prob) :
    T = len(obs)
    K = len(states)
    
    # initialize
    V = np.zeros((T, K))
    path = np.zeros((T, K), dtype = int) # optimal path
    
    # initialize with first observation
    V[0, :] = np.log(start_prob) + np.log(em_prob[:, obs[0]])
    
    # viterbi algorithm
    
    for t in range(1, T) :
        for j in range(K) :
            prob = V[t-1] + np.log(trans_prob[:, j]) + np.log(em_prob[j, obs[t]])
            V[t, j] = np.max(prob)
            path[t, j] = np.argmax(prob)
            
    # find optimal path through back tracking
    best_path = np.zeros(T, dtype = int)
    best_path[-1] = np.argmax(V[-1])
    
    for t in range(T-2, -1, -1) :
        best_path[t] = path[t+1, best_path[t+1]]
        
    return best_path, np.max(V[-1])

In [6]:
states = np.array([1, 2, 3])  # example of state space 
observations = np.array([0, 1, 2, 1, 0])  # observation index
start_prob = np.array([0.2, 0.4, 0.4])
trans_prob = np.array([
    [0.5, 0.2, 0.3],
    [0.3, 0.5, 0.2],
    [0.2, 0.3, 0.5]
])
em_prob = np.array([
    [0.5, 0.3, 0.2],
    [0.2, 0.5, 0.3],
    [0.3, 0.2, 0.5]
])

In [7]:
best_path, max_prob_log = viterbi(observations, states, start_prob, trans_prob, em_prob)
print("Best path:", best_path)
print("Max log probability of best path:", max_prob_log)

Best path: [1 1 1 1 0]
Max log probability of best path: -9.0925573363198


#### 1.1 Filtering and Smoothing with HMMs

금융 데이터는 전형적으로 잡음이 많기 때문에 잡음에서 시그널을 추출할 기법이 필요하다. 잡음을 줄이는 많은 기법이 있다. filtering은 잡음이 섞인 시그널에서 정보를 추출하는 일반적 방식이다. smoothing은 저빈도 요소는 통과하고 고빈도 요소는 약하게 되는 특별한 종류의 필터링이다. 필터링과 평활화는 각 시점에서 상태의 분포를 산출한다. 최대 우도 추정이 각 시점에서의 최적 추정 값에서의 최고 확률을 가진 상태를 선택하지만 이는 HMM에서 가장 좋은 경로를 구해주지 않는다. Baum-Welch Algorithm이 최적 상태의 최적 시퀀스뿐만 아니라 최적 상태 궤적을 찾고자 촬용될 수 있다는 것을 살펴봤다.

### 2. State-Space Models

HMM은 Gausian State-Space Model과 같은 클래스에 속한다. 이들은 HMM의 연속 잠재 상태 버전인 **Kalman Filter**로 알려져 있다. 상태 전이 확률 $p(s_t | s_{t-1})$은 다음과 같이 어떤 결정적 함수에 대해 결정적 부분과 잡음으로 분해할 수 있다.

$$s_t = F_t(s_{t-1}) + \epsilon_t$$

여기서 $\epsilon_t$는 0 평균의 IID 잡음이다. 유사하게 출력 확률 $p(y_t | s_t)$는 다음과 같이 분해될 수 있다. 

$$y_t = G_t(s_t) + \xi_t$$

여기서 관측 오차는 0, 평균의 IID 잡음이다. $F_t$와 $G_t$가 선형이고, 시간에 대해 독립적이라면 다음을 얻는다.

$$
\begin{align}
s_t = & As_{t-1} + \epsilon_{t} \\
y_t = & Cs_t + \xi_t
\end{align}
$$

여기서 $A$는 상태 전이 행렬이고, $C$는 관측 행렬이다. 완결성을 위해 칼만 필터와 일변량 RNN을 비교한다. 관측이 예측 변수 $x_t$이고 hidden variable이 $s_t$이면 다음을 얻는다.

$$
\begin{align}
s_t =& F(s_{t-1}, y_t) := \sigma(As_{t-1} + By_t)\\
y_t =& Cs_t + \xi_t
\end{align}
$$

여기서 단순화를 위해 편향에 대한 항은 전부 무시한다. 따라서 RNN의 상태 방정식은 이전 상태와 관측 모두의 비선형 함수이고, 잡음이 존재하지 않는다는 점에서 Kalman Filter와 다른다.

### 3. Particle Filtering

Kalman Filter는 상태를 다변량 가우시안 분포 $\mathcal{N}(m, P)$의 적률로 표현한다. 이 방법은 상태가 Gaussian, 또는 진정한 분포가 Gaussian에 매우 근사할 때 적절하다. 예를 들어 분포가 binomial이면 어떻게 될까? 아마도 가장 간단한 방법은 목표로 하는 분포에서 샘플링된 데이터 포인트로 이중 모드 분포를 포함한 대략 어떠한 분포를 근사하는 것이다. 이 데이터 포인트를 particle이라 한다.

particle을 더 많이 가질수록 목표 분포에 더욱 근접할 수 있다. 경험적인 근사 분포가 histogram으로 주어진다. particle은 예제에서와 같이 일변량일 필요는 없다. 다변량 분포를 근사한다면 다변량일 것이다. 또한 우리의 예제에서는 모든 particle가 동일 가중치를 갖지만, 더 일반적으로 가중치가 상이한 **weighted particle**을 다룰 수 있다. 

이러한 설정은 *particle filtering algorithms*로 알려진 algorithm군을 탄생하게 했다. 그 중 가장 일반적인 것이 Sequential Importance Resampling이다.

#### 3.1 Sequential Importance Resampling

1. Initialize : 시점 $t = 0$에서 초기 분포 $\tau_0$으로부터 $M$개의 IID 샘플을 추출한다. 또한 $M$개의 정규화된 가중치를 동일한 값 $\frac{1}{M}$로 초기화한다. $i = 1, \dots, M$에 대해 샘플은 $\hat{\mathbf{x}}_{0|0}^{(i)}$로 표기하고 정규화된 가중치는 $\lambda_0^{(i)}$로 표기한다.
2. Recursive Step : 시점 $t = 1, \dots, T$에서 $\left( \hat{\mathbf{x}}_{t-1 | t-1}^{(i)}\right)_{i=1, \dots, M}$을 시점 $t-1$에서 생성된 입자라 하자.
3. Importance Sampling : $i = 1, \dots, M$에 대해 Markov Kernel $\tau_t \left( \cdot \vert \hat{\mathbf{x}}_{t-1 | t-1}^{(i)}\right)$에서 $\hat{\mathbf{x}}_{t-1 | t-1}^{(i)}$을 샘플링한다. $i=1, \dots, M$에 대해 관측 밀도를 사용해 다음의 비정규화된 가중치를 계산한다.

$$\omega_t^{(i)} := \lambda_{t-1}^{(i)} \cdot p \left( \mathbf{y}_t \vert \hat{\mathbf{x}}_{t | t-1}^{(i)}\right)$$ 

그리고 리샘플링 전의 정규화된 가중치를 구한다.

$$^\text{br} \lambda_t^{(i)} := \frac{\omega_t^{(i)}}{\sum_{k=1}^M \omega_t^{(k)}}$$

4. Resampling : $i = 1, \dots, M$에 대해 적절한 Resampling algorithm을 사용해 다음 혼합 분포로부터 $\hat{\mathbf{x}}_{t|t}^{(i)}$를 샘플링한다.

$$\sum_{k=1}^M  \sideset{^\text{br}} \lambda_t^{(k)} \delta \left( \mathbf{x} - \mathbf{x}_{t | t-1}^{(k)}\right)$$

여기서 $\delta(\cdot)$은 Direc Delta 일반화 함수이며 resampling 후의 정규화된 가중치 $\lambda_t^{(i)}$를 적절히 설정한다. (가장 일반적인 resampling algorithm에 대해 이는 $\lambda_t^{(i)} := \frac{1}{M}$을 의미한다) 비공식적으로 SIR은 유전 알고리즘 성격을 일부 갖는다. 우도 $p(\mathbf{y}_t | \hat{\mathbf{x}}_{t|t-1}^{(i)})$을 기반으로 가장 성공적인 입자의 가중치를 증가해 이들이 resampling 단계에서 잘 추출되게 한다.

resampling 단계는 모든 가중치가 하나의 포인트에 집중되는 입자들의 축퇴를 피하고자 도입된다. 가장 일반적인 resampling method는 곧 살펴볼 소위 multinomial resampling이다.

#### 3.2 Multinomial Resampling

3.1절에서 resampling 이전에 계산된 정규화된 가중치 $\sideset{^\text{br}}\lambda_t^{(1)}, \dots, \sideset{^\text{br}}\lambda_t^{(M)}$을 사용한다.

a. $i = 1, \dots, M$에 대해 다음 누적을 계산한다. 

$${}^\text{br} \Lambda_{t}^{(i)} = \sum_{k=1}^i \sideset{^\text{br}} \lambda_t^{(k)}$$

그에 따라 계산해 $^\text{br} \Lambda_t^{(M)} = 1$을 얻는다.

b. $\mathcal{u}(0,1)$에서 $M$ random sample $u_1, u_2, \dots, u_M$을 생성한다.

c. 각 $i = 1, \dots, M$에 대해 $u_i \in \left[ {}^\text{br}\Lambda_{t}^{(j)} , {}^\text{br}\Lambda_{t}^{(j+1)}\right]$가 되는 입자 $\hat{\mathbf{x}}_{t|t}^{(i)} = \hat{\mathbf{x}}_{t|t-1}^{(j)}$를 선택한다. 여기서 $j \in \left\{ 1, 2, \dots, M-1\right\}$이다.

그러면 기존 집합 $\hat{\mathbf{x}}_{t|t-1}^{(1)}, \dots, \hat{\mathbf{x}}_{t|t-1}^{(M)}$에서 샘플링된 $M$개의 새로운 입자  $\hat{\mathbf{x}}_{t|t}^{(1)}, \dots, \hat{\mathbf{x}}_{t|t}^{(M)}$를 갖게 되며, 기존 입자 중 일부는 사라지는 반면 다른 입자들은 여러 번 나오게 된다. 각 $i = 1, \dots, M$에 대해 $\hat{\mathbf{x}}_{t|t-1}^{(i)}$가 입자의 resample set에 나타나는 횟수는 replication factor $N_t^{(i)}$로 알려져 있다.

resampling 후 정규화 가중치를 $\lambda_t^{(i)} := \frac{1}{M}$로 설정한다. 이 알고리즘을 확률 $\sideset{^\text{br}}\lambda_t^{(1)}, \dots, \sideset{^\text{br}}\lambda_t^{(M)}$을 가진 다항 분포로부터 복제 배율 $N_t^{(1)}, \dots, N_t^{(M)}$을 각각 샘플링하는 것으로 볼 수 있다. 따라서 이 방법의 이름도 나왔다.